In [117]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import re
from collections import defaultdict

In [118]:
#Use one ./ if running from run_notebooks.py
#Use two ../if running directly from Jupyter Notebooks
file_beg = '../NHANES-Downloader/data/csv_data/'

In [119]:
#Import all the files
files1 = glob.glob(file_beg+'1999-2000/*/*.csv')
files2 = glob.glob(file_beg+'2001-2002/*/*.csv')
files3 = glob.glob(file_beg+'2003-2004/*/*.csv')
files4 = glob.glob(file_beg+'2005-2006/*/*.csv')
files5 = glob.glob(file_beg+'2007-2008/*/*.csv')
files6 = glob.glob(file_beg+'2009-2010/*/*.csv')
files7 = glob.glob(file_beg+'2011-2012/*/*.csv')
files8 = glob.glob(file_beg+'2013-2014/*/*.csv')
files9 = glob.glob(file_beg+'2015-2016/*/*.csv')
files10 = glob.glob(file_beg+'2017-2018/*/*.csv')

In [120]:
#Add files into a list
file_list = [files1, files2, files3, files4, files5, 
          files6, files7, files8, files9, files10]

In [121]:
#Add files into a list of sorted and dictionaries
file_list_dict = []
for x in file_list:
    x.sort()
    file_list_dict.append(dict(enumerate(x)))

In [122]:
alq_indx = [60, 76, 90, 88, 86, 92, 98, 127, 96, 113]

In [123]:
dfs = defaultdict(int)
#dfs[1].show
for i in range(0,len(alq_indx)):
    dfs[i] = pd.read_csv(file_list_dict[i][alq_indx[i]])

## Functions: Recategorize values, Count Values, Drop Rows

In [124]:
#Recategorize function
def recategorize(df, name, replace_dict):
    df[name].replace(
    to_replace=replace_dict,
    inplace=True
)
    
#Count values function
def count_vals(df, name):
    df_count = df.groupby(name)['SEQN'].nunique()
    print(df_count,"\n\n","NaN: ", df[name].isnull().sum())
    
#Drop rows that include certain values
def drop_rows(df, name, val_list):
    df.drop(df[df[name].isin(val_list)].index, inplace=True)

## Important Variables

In [125]:
#1999-2018
descr = ['SEQN', '12DRINKS (YR)']
var_names = ["SEQN", "ALQ101"]

In [126]:
#To display all columns in Jupyter Notebooks
#pd.set_option('display.max_columns', 500)

## Rename 1999-2000 and 2001-2002 columns

In [127]:
#Rename ALQ100 to ALQ101
dfs[0].rename(columns={'ALQ100': 'ALQ101'}, inplace=True)
#Rename ALD100 to ALQ101
dfs[1].rename(columns={'ALD100': 'ALQ101'}, inplace=True)
#Rename ALQ111 to ALQ101
dfs[9].rename(columns={'ALQ111': 'ALQ101'}, inplace=True)

## Make a copy of the dataframes

In [128]:
#1999-20016
df0 = dfs[0][var_names].copy() #1999-2000
df1 = dfs[1][var_names].copy() #2001-2002
df2 = dfs[2][var_names].copy() #2003-2004
df3 = dfs[3][var_names].copy() #2005-2006
df4 = dfs[4][var_names].copy() #2007-2008
df5 = dfs[5][var_names].copy() #2009-2010
df6 = dfs[6][var_names].copy() #2011-2012
df7 = dfs[7][var_names].copy() #2013-2014
df8 = dfs[8][var_names].copy() #2015-2016
df9 = dfs[9][var_names].copy() #2017-2018

## Recategorize values

In [129]:
#No recategorizing needed

## Rename columns 1999 - 2016

In [130]:
#No renaming needed

## Append years 1999 - 2018

In [131]:
years = ["1999-2000","2001-2002","2003-2004","2005-2006","2007-2008", 
        "2009-2010","2011-2012","2013-2014","2015-2016","2017-2018"]

In [132]:
frames = [df0, df1, df2, df3, df4, df5, df6, df7, df8, df9]

In [133]:
#Add years as a column
for i, df in enumerate(frames):
    df["Year"] = years[i]

In [134]:
result_1999_2018 = pd.concat(frames, keys = years)

In [135]:
result_1999_2018_cleaned = result_1999_2018.copy()

In [136]:
result_1999_2018_cleaned

SEQN  ALQ101       Year
1999-2000 0          2.0     1.0  1999-2000
          1          5.0     1.0  1999-2000
          2          7.0     2.0  1999-2000
          3         10.0     1.0  1999-2000
          4         12.0     1.0  1999-2000
...                  ...     ...        ...
2017-2018 5528  102949.0     2.0  2017-2018
          5529  102952.0     1.0  2017-2018
          5530  102953.0     1.0  2017-2018
          5531  102954.0     2.0  2017-2018
          5532  102956.0     1.0  2017-2018

[53559 rows x 3 columns]

## Count values for each column

In [137]:
col_names = var_names

In [138]:
len(col_names)

2

In [139]:
#ALQ - Ever told you had high blood pressure
count_vals(result_1999_2018, col_names[1])

ALQ101
1.0    34806
2.0    13910
9.0       33
Name: SEQN, dtype: int64 

 NaN:  4810


In [140]:
len(result_1999_2018)

53559

## Remove missing values:

In [141]:
drop_rows(result_1999_2018_cleaned, col_names[1], [np.nan, 7, 9])

## See if missing values have been correctly removed:

In [142]:
count_vals(result_1999_2018_cleaned, col_names[1])

ALQ101
1.0    34806
2.0    13910
Name: SEQN, dtype: int64 

 NaN:  0


In [143]:
before = len(result_1999_2018)
before

53559

In [144]:
after = len(result_1999_2018_cleaned)
after

48716

In [145]:
(before-after)/after

0.09941292388537647

## MongoDB Insertion

In [146]:
#Import MongoClient
from pymongo import MongoClient

#Create a MongoClient to run the MongoDB instance
client = MongoClient("localhost", 27017)

In [147]:
#Connect to existing database
db = client.NHANES
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'NHANES')

In [148]:
db.list_collection_names()

['alq', 'bmx']

In [149]:
#Creating a collection
alq = db.alq

In [150]:
#If collections exist, then drop
if 'alq' in db.list_collection_names():
    alq.drop()
    db.list_collection_names()

## Create new collection to input into database

In [151]:
result_1999_2018_cleaned[:3]

SEQN  ALQ101       Year
1999-2000 0   2.0     1.0  1999-2000
          1   5.0     1.0  1999-2000
          2   7.0     2.0  1999-2000

In [152]:
#Set SEQN as _id (Primary Key)
result_1999_2018_cleaned.rename(columns= {'SEQN':'_id'}, inplace=True)

In [153]:
#Dataframe to dictionary
alq_dict = result_1999_2018_cleaned.to_dict(orient='records')

In [154]:
alq_dict[0]

{'_id': 2.0, 'ALQ101': 1.0, 'Year': '1999-2000'}

In [155]:
#Insert collection
alq.insert_many(alq_dict)

In [156]:
db.list_collection_names()

['alq', 'bmx']